<a href="https://colab.research.google.com/github/Arju-Jawla/Wind_Speed/blob/main/WIND_FIELD_04_VIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installing Required Libraries

In [ ]:
!pip install xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 13.7 MB/s eta 0:00:00


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

## 2. Loading the data

In [ ]:
file_path = '/content/drive/MyDrive/Project Learnings/Data/WIND FIELD RELATED 1923/ERA5_merged_6hrly_daily_2019-23.nc'

In [ ]:
df = xr.open_dataset(file_path)

In [ ]:
df

<xarray.Dataset> Size: 3GB
Dimensions:         (valid_time: 7304, latitude: 121, longitude: 81)
Coordinates:
    expver          (valid_time) object 58kB ...
  * valid_time      (valid_time) datetime64[ns] 58kB 2019-01-01 ... 2023-12-3...
  * latitude        (latitude) float64 968B 30.0 29.75 29.5 ... 0.5 0.25 0.0
  * longitude       (longitude) float64 648B 80.0 80.25 80.5 ... 99.75 100.0
    number          int32 4B ...
Data variables:
    u10             (valid_time, latitude, longitude) float32 286MB ...
    v10             (valid_time, latitude, longitude) float32 286MB ...
    blh             (valid_time, latitude, longitude) float32 286MB ...
    cape            (valid_time, latitude, longitude) float32 286MB ...
    d2m             (valid_time, latitude, longitude) float32 286MB ...
    fsr             (valid_time, latitude, longitude) float32 286MB ...
    geo             (valid_time, latitude, longitude) float32 286MB ...
    msl             (valid_time, latitude, longitude) float32 286MB ...
    t2m             (valid_time, latitude, longitude) float32 286MB ...
    wind_magnitude  (valid_time, latitude, longitude) float32 286MB ...
    RH              (valid_time, latitude, longitude) float32 286MB ...
    spg             (valid_time, latitude, longitude) float32 286MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-12T10:19 GRIB to CDM+CF via cfgrib-0.9.1...

## 3.Data Preprocessing

In [ ]:
#  Select the relevant variables
parameters = ['spg', 't2m', 'geo', 'blh', 'fsr', 'RH', 'cape']
data_vars = df[parameters]

In [ ]:
# Reshape the data into a 2D DataFrame (time, lat, lon → rows, parameters → columns)
# Flatten the data
data_flattened = {var: data_vars[var].values.flatten() for var in parameters}

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data_flattened)

In [ ]:
# Handle missing values (drop rows with NaN values)
df_cleaned = df.dropna()

In [ ]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cleaned)

## 4. VIF

In [ ]:
# Compute Variance Inflation Factor (VIF)
# VIF requires no constant column in the data
vif_data = pd.DataFrame()
vif_data['Feature'] = parameters
vif_data['VIF'] = [variance_inflation_factor(scaled_data, i) for i in range(scaled_data.shape[1])]

In [ ]:
# Display the VIF values
print(vif_data)

  Feature       VIF
0     spg  2.016999
1     t2m  5.607886
2     geo  6.159995
3     blh  1.792786
4     fsr  1.234054
5      RH  1.587886
6    cape  1.270285


## 5. Save the results

In [ ]:
# Save the VIF data to a CSV file
output_file = "/content/drive/MyDrive/Project Learnings/Data/WIND FIELD RELATED 1923/vif_results1.csv"
vif_data.to_csv(output_file, index=False)

print(f"VIF data saved to {output_file}")

VIF data saved to /content/drive/MyDrive/Project Learnings/Data/WIND FIELD RELATED 1923/vif_results1.csv
